<a href="https://colab.research.google.com/github/IfeakanduBenedict/pediatric-pneumonia-ml-england/blob/main/Pediatric_Pneumonia_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Topic: Predicting Pediatric Pneumonia Hospitalisation Rates in English Local Authorities**
   
## Research Questions

**PRIMARY**: To what extent can machine learning models predict emergency pneumonia hospitalisation rates among children and young people under 19 across English Local Authorities using socioeconomic deprivation, demographic vulnerability, and healthcare utilisation indicators?

**SECONDARY**: Which socioeconomic, demographic, and healthcare-related factors contribute most to geographic inequalities in pneumonia hospitalisation rates among children and young people under 19 across England?


# **Notebook Setup and Installation of Needed Library**

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical analysis
from scipy import stats
from scipy.stats import pearsonr, spearmanr

# Machine Learning - Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler

# Import Variance Inflation Factor (VIF) to detect multicollinearity
# between predictor variables during feature evaluation.
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Import hyperparameter tuning tools for model optimization
# GridSearchCV performs exhaustive parameter search,
# while RandomizedSearchCV samples parameter combinations for faster exploration.
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Machine Learning - Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

# Import statistical tests for regression assumptions validation
# shapiro, normaltest: Test normality of residuals
# het_breuschpagan: Test homoscedasticity (constant variance)
# durbin_watson: Test independence (no autocorrelation)
from scipy.stats import shapiro, normaltest
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

# Import statsmodels for adding constant (required by Breusch-Pagan test)
import statsmodels.api as sm

# Machine Learning - Metrics
from sklearn.metrics import (mean_squared_error, mean_absolute_error,
                             r2_score, mean_absolute_percentage_error)

# Model Interpretation
import shap
from sklearn.inspection import permutation_importance

# Model Persistence
import joblib
import json
from datetime import datetime

# For handling file paths
import os

# Settings
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Configure plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
sns.set_context("notebook", font_scale=1.1)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("✓ All libraries imported successfully!")
print(f"✓ Random seed set to: {RANDOM_STATE}")

✓ All libraries imported successfully!
✓ Random seed set to: 42


# **Connecting Google Drive**

In [ ]:
# Import and mount Google Drive.
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Navigating to the dataset folder in Google Drive
os.chdir('/content/drive/MyDrive/')

## **Loading The Dataset**

In [ ]:
# Load the dataset
pneumonia_data = pd.read_csv('pediatric_pneumonia_england_LA_dataset_2023_2024.csv')

# **Dataset Overview**

In [ ]:
# Displaying the first five rows.
pneumonia_data.head()

,Area_Code,Area_Name,Pneumonia_Rate_per_100k,IMD_Score,Child_Poverty_Pct,Fuel_Poverty_Pct,Free_School_Meals_Pct,Pop_0_4_Years_Pct,Population_Density,Ethnic_Minority_Pct,Overcrowded_HH_Pct,Infant_Mortality_Rate,Birth_Rate_per_1000,Emergency_Admissions_U18,Respiratory_Admissions_0_4,Emergency_Admissions_0_4
0,E07000223,Adur,592.0,17.59,15.3,9.69,38.39,113.75,1538.65,11.07,25.96,4.60,48.40,55.79,178.75,118.62
1,E07000032,Amber Valley,405.2,17.97,21.3,11.94,37.06,84.52,478.30,4.96,29.53,3.95,50.28,39.93,126.77,86.21
2,E07000224,Arun,681.4,18.64,17.4,9.52,34.68,154.41,752.88,11.35,28.75,2.67,47.68,73.94,224.59,167.74
3,E07000170,Ashfield,748.9,26.31,28.2,12.27,34.98,74.14,1160.12,8.18,42.70,5.33,50.73,70.80,229.83,174.23
4,E07000105,Ashford,911.8,18.55,17.1,9.86,34.15,118.05,233.79,17.38,30.65,5.94,53.30,74.18,282.00,180.35


In [ ]:
# Dispalying the last five rows.
pneumonia_data.tail()

,Area_Code,Area_Name,Pneumonia_Rate_per_100k,IMD_Score,Child_Poverty_Pct,Fuel_Poverty_Pct,Free_School_Meals_Pct,Pop_0_4_Years_Pct,Population_Density,Ethnic_Minority_Pct,Overcrowded_HH_Pct,Infant_Mortality_Rate,Birth_Rate_per_1000,Emergency_Admissions_U18,Respiratory_Admissions_0_4,Emergency_Admissions_0_4
277,E07000229,Worthing,713.9,17.01,14.9,10.27,38.33,119.26,3444.72,14.96,25.69,1.85,45.12,63.00,228.58,139.14
278,E07000238,Wychavon,522.7,15.77,16.6,15.01,34.94,79.74,202.77,9.39,23.21,3.31,50.15,58.77,175.44,127.59
279,E07000128,Wyre,950.2,20.86,19.6,11.50,36.55,104.52,407.30,4.26,39.32,5.33,46.17,100.17,324.00,244.57
280,E07000239,Wyre Forest,519.2,22.44,24.2,15.82,32.80,65.88,523.56,7.51,30.27,5.95,49.88,54.49,175.67,115.28
281,E06000014,York,1061.4,11.73,15.8,12.27,38.06,119.62,750.60,12.75,28.20,3.75,30.95,99.24,322.97,241.03
